# 1. Import packages and load json file

import the necessary packages:

* import nltk
* from nltk.stem import WordNetLemmatizer
* lemmatizer = WordNetLemmatizer()
* import json
* import pickle


* import numpy as np
* from keras.models import Sequential
* from keras.layers import Dense, Activation, Dropout
* from keras.optimizers import SGD
* import random


In [18]:
import nltk
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [19]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [20]:
from google.colab import files
uploaded = files.upload()

Saving intents.json to intents (1).json


In [21]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 2. Preprocess data

Preprocess the data first before making a machine learning model. Perform **tokenization** - it is the process of breaking the whole text into small parts (words).

The following code iterates through the patterns in the intents.json file and tokenizes the sentence using nltk.word_tokenize() function and then appends eacch word in the words list.


In [23]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [24]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

The following code lemmatize each word and removes duplicate words from the list. 

**Lemmatizing** is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which will be use in predicting. 

In [25]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

11 documents
3 classes ['goodbye', 'greeting', 'thanks']
19 unique lemmatized words ["'s", 'anyone', 'are', 'bye', 'day', 'good', 'goodbye', 'hello', 'helpful', 'hi', 'how', 'is', 'later', 'see', 'thank', 'thanks', 'that', 'there', 'you']


# 3. Create training data

Create the training data in which we will provide the input and output - input will be the pattern, output will be the class (*tags* in the intents.json file) the input pattern belongs to. 

Since machine learning cannot directly work with categorical data, convert to numeric data.

In [26]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence

for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


# 4. Build machine learning model

After creating training data, build a deep neaural network that has 3 layers. The following code uses Keras' sequential API. 

The model is trained for 200 epochs, achieving 100% accuracy on the model. 


In [27]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
4/4 [==============================] - 1s 3ms/step - loss: 1.0285 - accuracy: 0.5393
Epoch 2/200
4/4 [==============================] - 0s 3ms/step - loss: 0.5962 - accuracy: 0.9456
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 0.2307 - accuracy: 1.0000
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0958 - accuracy: 1.0000
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0555 - accuracy: 1.0000
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 7/200
4/4 [==============================] - 0s 4ms/step - loss: 0.0175 - accuracy: 1.0000
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 9/200
4/4 [==============================] - 0s 5ms/step - loss: 0.0046 - accuracy: 1.0000
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 11/200
4/4 [===========

# 5. Predicting sentences and getting a response from the user

loaded the trained model saved as *chatbot_model.h5* on the previous cell.

The model trained (so far) will only tell the class the response of the bot belongs to, so implement some functions that'll identify the class and then retrieve a random response from the list of responses provided in the intents.json. 

Import necessary package:

* from keras.models import load_model



In [28]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

The following functions will perform text preprocessing and then predicts the class.

In [29]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [30]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [31]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

The following functions will get a random response from the list of intents.

In [32]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [33]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res